In [1]:
import pandas as pd
from sqlalchemy import create_engine
from config import user, password
import datetime as dt

### Extract CSVs into DataFrames

In [2]:
flights_file = "Resources/flights.csv"
flights_df = pd.read_csv(flights_file, dtype = {'SCHEDULED_DEPARTURE':str})
flights_df.head()

/Users/williamharris/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,...,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY
0,2015,1,1,4,AS,98,N407AS,ANC,SEA,0005,...,408.0,-22.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
1,2015,1,1,4,AA,2336,N3KUAA,LAX,PBI,0010,...,741.0,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
2,2015,1,1,4,US,840,N171US,SFO,CLT,0020,...,811.0,5.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
3,2015,1,1,4,AA,258,N3HYAA,LAX,MIA,0020,...,756.0,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,2015,1,1,4,AS,135,N527AS,SEA,ANC,0025,...,259.0,-21.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
airports_file = "Resources/airports.csv"
airports_df = pd.read_csv(airports_file).drop(['AIRPORT', 'STATE', 'COUNTRY', 'LATITUDE', 'LONGITUDE'], axis=1)
airports_transformed=airports_df.rename(columns={"IATA_CODE": "airport_code",
                                        "CITY": "city"}) 
airports_transformed.head()


,airport_code,city
0,ABE,Allentown
1,ABI,Abilene
2,ABQ,Albuquerque
3,ABR,Aberdeen
4,ABY,Albany


In [21]:
weather_file = "Resources/weather_description.csv"
weather_df = pd.read_csv(weather_file)
weather_df.head()

,datetime,Vancouver,Portland,San Francisco,Seattle,Los Angeles,San Diego,Las Vegas,Phoenix,Albuquerque,...,Philadelphia,New York,Montreal,Boston,Beersheba,Tel Aviv District,Eilat,Haifa,Nahariyya,Jerusalem
0,2012-10-01 12:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,haze,NaN,NaN,NaN
1,2012-10-01 13:00:00,mist,scattered clouds,light rain,sky is clear,mist,sky is clear,sky is clear,sky is clear,sky is clear,...,broken clouds,few clouds,overcast clouds,sky is clear,sky is clear,sky is clear,haze,sky is clear,sky is clear,sky is clear
2,2012-10-01 14:00:00,broken clouds,scattered clouds,sky is clear,sky is clear,sky is clear,sky is clear,sky is clear,sky is clear,sky is clear,...,broken clouds,few clouds,sky is clear,few clouds,sky is clear,sky is clear,broken clouds,overcast clouds,sky is clear,overcast clouds
3,2012-10-01 15:00:00,broken clouds,scattered clouds,sky is clear,sky is clear,sky is clear,sky is clear,sky is clear,sky is clear,sky is clear,...,broken clouds,few clouds,sky is clear,few clouds,overcast clouds,sky is clear,broken clouds,overcast clouds,overcast clouds,overcast clouds
4,2012-10-01 16:00:00,broken clouds,scattered clouds,sky is clear,sky is clear,sky is clear,sky is clear,sky is clear,sky is clear,sky is clear,...,broken clouds,few clouds,sky is clear,few clouds,overcast clouds,sky is clear,broken clouds,overcast clouds,overcast clouds,overcast clouds


In [22]:
weather_transformed=weather_df[['datetime','Atlanta','Chicago','Kansas City','New York','San Francisco']]


weather_transformed = weather_transformed.rename(columns={"Atlanta": "atlanta", 
                                                          "Chicago": "chicago", 
                                                          "Kansas City": "kansas_city", 
                                                          "New York": "new_york", 
                                                          "San Francisco": "san_francisco"})


weather_transformed=weather_transformed[weather_transformed["datetime"]>="2015-01-01"]
weather_transformed=weather_transformed[weather_transformed["datetime"]<="2015-12-31"]
weather_transformed.head()

,datetime,atlanta,chicago,kansas_city,new_york,san_francisco
19716,2015-01-01 00:00:00,sky is clear,sky is clear,scattered clouds,sky is clear,sky is clear
19717,2015-01-01 01:00:00,sky is clear,sky is clear,scattered clouds,sky is clear,sky is clear
19718,2015-01-01 02:00:00,sky is clear,sky is clear,scattered clouds,sky is clear,sky is clear
19719,2015-01-01 03:00:00,sky is clear,sky is clear,scattered clouds,sky is clear,sky is clear
19720,2015-01-01 04:00:00,sky is clear,sky is clear,scattered clouds,sky is clear,sky is clear


### Transform flights DataFrame

In [23]:
# Create a filtered dataframe from specific columns
flights_cols = ["YEAR", "MONTH", "DAY", "ORIGIN_AIRPORT","SCHEDULED_DEPARTURE", "DEPARTURE_DELAY", "CANCELLED","FLIGHT_NUMBER" ]
flights_transformed= flights_df[flights_cols].copy()


# select 

flights_transformed=flights_transformed[flights_transformed["ORIGIN_AIRPORT"].isin(["ORD", "JKF", "SFO", "ATL", "MCI"])]
flights_transformed=flights_transformed[flights_transformed["DEPARTURE_DELAY"]>60.0]


flights_transformed.head()

,YEAR,MONTH,DAY,ORIGIN_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_DELAY,CANCELLED,FLIGHT_NUMBER
588,2015,1,1,SFO,0630,76.0,0,577
1137,2015,1,1,ORD,0719,82.0,0,5299
1284,2015,1,1,ORD,0730,65.0,0,1070
1379,2015,1,1,SFO,0740,171.0,0,240
1894,2015,1,1,ORD,0815,100.0,0,6378


In [12]:
date=[]
for index, row in flights_transformed.iterrows():
    year= row["YEAR"]
    month= row["MONTH"]
    day= row["DAY"]
    time= row["SCHEDULED_DEPARTURE"]
    date.append(pd.to_datetime(f"{year}-{month}-{day}-{time}").round('60min'))

In [24]:
flights_transformed['datetime'] = date

flights_transformed = flights_transformed.rename(columns={"ORIGIN_AIRPORT": "airport_code", 
                                                          "DEPARTURE_DELAY":"departure_delay",
                                                          "FLIGHT_NUMBER":"flight_number",
                                                          "CANCELLED": 'cancellation'
                                                          })

flights_transformed=flights_transformed.merge(airports_transformed, left_on='airport_code', right_on='airport_code')

flights_transformed=flights_transformed.drop(['YEAR','MONTH', 'DAY','SCHEDULED_DEPARTURE' ], axis=1)
flights_transformed=flights_transformed[['datetime','airport_code', 'city', 'flight_number', 'departure_delay', 'cancellation']]

flights_transformed.head()

,datetime,airport_code,city,flight_number,departure_delay,cancellation
0,2015-01-01 06:00:00,SFO,San Francisco,577,76.0,0
1,2015-01-01 08:00:00,SFO,San Francisco,240,171.0,0
2,2015-01-01 08:00:00,SFO,San Francisco,414,86.0,0
3,2015-01-01 09:00:00,SFO,San Francisco,5445,96.0,0
4,2015-01-01 09:00:00,SFO,San Francisco,164,193.0,0


### Create database connection

In [25]:
connection_string = "postgres:postgres@localhost:5432/ETL Project"
engine = create_engine(f'postgresql://{connection_string}')

In [26]:
# Confirm tables
engine.table_names()

['flights', 'weather']

### Load DataFrames into database

In [27]:
flights_transformed.to_sql(name='flights', con=engine, if_exists='append', index=True)

In [28]:
weather_transformed.to_sql(name='weather', con=engine, if_exists='append', index=True)